In [3]:
import azureml.core
from azureml.core.compute import ComputeTarget, BatchAiCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import EnvironmentDefinition
from azureml.train.estimator import Estimator
from azureml.core.workspace import Workspace
from azureml.core import Experiment
from azureml.train.widgets import RunDetails

import json
import sys
import os
import shutil

sys.path.append('src')

from registry import azure_container_registry_for
import batch_ai

In [4]:
print("SDK version:", azureml.core.VERSION)

SDK version: 0.1.65


In [5]:
ws = Workspace.from_config(path='configs/azml_config.json')

Found the config file in: /workspace/configs/azml_config.json


In [6]:
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: mstest
Azure region: eastus2
Subscription id: edf507a2-6235-46c5-b560-fd463ba2e771
Resource group: msazmlrg


In [7]:
registry_name = ws.get_details()['containerRegistry'].split('/')[-1]
registry_name

'mstestacrhomithdz'

In [8]:
azr = azure_container_registry_for(ws.resource_group, registry_name, subscription_id=ws.subscription_id)

In [9]:
compute_target = batch_ai.cluster(ws, "gpucluster")

In [10]:
from azureml.core.compute import ComputeTarget, BatchAiCompute

In [11]:
project_folder = './batchai_test'
os.makedirs(project_folder, exist_ok=True)

In [43]:
%%writefile test_batch.py
import subprocess
from pprint import pprint

if __name__=='__main__':
    results = subprocess.run(['printenv'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    pprint(results.stdout.decode('ascii'))
    results = subprocess.run(['df', '-h'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    pprint(results.stdout.decode('ascii'))
    results = subprocess.run(['pwd'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    pprint(results.stdout.decode('ascii'))
    results = subprocess.run(['ls', '-ltra', 'azmlblob'], stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    pprint(results.stdout.decode('ascii'))

Overwriting test_batch.py


In [13]:
%run test_batch.py

('CUDNN_VERSION=7.0.5.15-1+cuda9.0\n'
 'HOSTNAME=77cc5dccb9af\n'
 'NVIDIA_REQUIRE_CUDA=cuda>=9.0\n'
 'TERM=xterm-color\n'
 'LIBRARY_PATH=/usr/local/cuda/lib64/stubs\n'
 'LC_ALL=en_US.UTF-8\n'
 'NVIDIA_VISIBLE_DEVICES=all\n'
 'LD_LIBRARY_PATH=/usr/local/nvidia/lib:/usr/local/nvidia/lib64\n'
 'NVIDIA_DRIVER_CAPABILITIES=compute,utility\n'
 'PATH=/opt/conda/envs/py3.6/bin:/opt/conda/envs/py3.6/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin\n'
 'PWD=/workspace\n'
 'ENV_NAME=py3.6\n'
 'LANG=en_US.UTF-8\n'
 'PYTORCH_VERSION=0.4.0\n'
 'CUDA_PKG_VERSION=9-0=9.0.176-1\n'
 'CUDA_VERSION=9.0.176\n'
 'SHLVL=1\n'
 'HOME=/root\n'
 'LANGUAGE=en_US:en\n'
 'NCCL_VERSION=2.2.13-1+cuda9.0\n'
 '_=/opt/conda/envs/py3.6/bin/jupyter\n'
 'JPY_PARENT_PID=1\n'
 'CLICOLOR=1\n'
 'PAGER=cat\n'
 'GIT_PAGER=cat\n'
 'MPLBACKEND=module://ipykernel.pylab.backend_inline\n')
('Filesystem      Size  Used Avail Use% Mounted on\n'
 'overlay          

In [44]:
shutil.copy('test_batch.py', project_folder)

'./batchai_test/test_batch.py'

In [45]:
env = EnvironmentDefinition()
env.python.user_managed_dependencies=True
env.docker.enabled = True
env.docker.gpu_support = True
env.docker.shared_volumes = True
env.docker.base_image = "azml"
env.docker.base_image_registry=azr

In [16]:
from azureml.core import Datastore

In [17]:
ws.get_details()

{'id': '/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourceGroups/msazmlrg/providers/Microsoft.MachineLearningServices/workspaces/mstest',
 'name': 'mstest',
 'location': 'eastus2',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'description': '',
 'friendlyName': 'mstest',
 'containerRegistry': '/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/msazmlrg/providers/microsoft.containerregistry/registries/mstestacrhomithdz',
 'keyVault': '/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/msazmlrg/providers/microsoft.keyvault/vaults/mstestkeyvaultsidoqlor',
 'applicationInsights': '/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/msazmlrg/providers/microsoft.insights/components/mstestinsightsvullbbcb',
 'batchaiWorkspace': '/subscriptions/edf507a2-6235-46c5-b560-fd463ba2e771/resourcegroups/msazmlrg/providers/microsoft.batchai/workspaces/mstestbatchai_nurisqhl',
 'identityPrincipalId': '5249feb8-f44e-4861-9dd4-da9cf0e09

In [22]:
default_ds=ws.get_default_datastore()

In [53]:
default_ds.

'azureml-filestore-8c170506-e529-4ebe-a306-a7789c1dbdbf'

In [31]:
ds = Datastore.register_azure_blob_container(workspace=ws, 
                                             datastore_name='azmlblob', 
                                             container_name='msazmlblob',
                                             account_name=default_ds.account_name, 
                                             account_key=default_ds.account_key,
                                             create_if_not_exists=True)

In [34]:
ds.upload('batchai_test', target_path='batchai_test', show_progress=True)

$AZUREML_DATAREFERENCE_7911b54468c34a479b6fc854267eb29d

In [36]:
ds_data = ds.path('batchai_test')

In [30]:
# ds = ws.get_default_datastore()
# print(ds.datastore_type, ds.account_name, ds.container_name)
# ds.upload(src_dir='./batchai_test', target_path='batchai_test')
# path_on_datastore = 'batchai_test'
# ds_data = ds.path(path_on_datastore)
# print(ds_data)

AzureFile msteststoragebnqmxwim azureml-filestore-8c170506-e529-4ebe-a306-a7789c1dbdbf


Target already exists. Skipping upload for batchai_test/test_batch.py
Target already exists. Skipping upload for batchai_test/.amlignore
Target already exists. Skipping upload for batchai_test/aml_config/docker.runconfig
Target already exists. Skipping upload for batchai_test/aml_config/local.runconfig
Target already exists. Skipping upload for batchai_test/aml_config/conda_dependencies.yml
Target already exists. Skipping upload for batchai_test/aml_config/project.json


$AZUREML_DATAREFERENCE_dcf983a37fb441ef8cb8d4ff7d40ab93


In [46]:
estimator = Estimator(source_directory=project_folder, 
                    compute_target=compute_target,
                    entry_script='test_batch.py',
                    environment_definition=env,
                     inputs=[ds_data.as_download()])

In [47]:
experiment_name='batchai-test'
experiment = Experiment(ws, name=experiment_name)

In [48]:
run = experiment.submit(estimator)

In [49]:
print(json.dumps(run.get_details(), indent=4))

{
    "runId": "batchai-test_1539601806656",
    "target": "gpucluster",
    "status": "Running",
    "startTimeUtc": "2018-10-15T11:10:08.302723Z",
    "properties": {
        "azureml.runsource": "experiment",
        "ContentSnapshotId": "4cf97da6-880e-4bd8-a166-ad0009835e60"
    },
    "runDefinition": {
        "Script": "test_batch.py",
        "Arguments": [],
        "Framework": 0,
        "Target": "gpucluster",
        "DataReferences": {
            "bdcd8374a6d0490ab035824868d96ea9": {
                "DataStoreName": "azmlblob",
                "Mode": "Download",
                "PathOnDataStore": "batchai_test",
                "PathOnCompute": null,
                "Overwrite": false
            }
        },
        "JobName": null,
        "AutoPrepareEnvironment": true,
        "MaxRunDurationSeconds": null,
        "Environment": {
            "Python": {
                "InterpreterPath": "python",
                "UserManagedDependencies": true,
                "C

In [50]:
RunDetails(run).show()

_UserRun()